We practiced web scraping when all the information is in a single table of a single page in a site. What happens when we want to scrape information from multiple pages?

Go to https://www.imdb.com/search/title/ and enter the following parameters, leaving all other fields blank or with its default value:

- Title Type: Feature film

- Release date: From 1990 to 1992

- User Rating: 7.5 to "-"

The page you get should be familiar. There's a list with movies and each movie has its title, release year, crew, etc. You could inspect the page and build the code to collect the date.

However, the results we obtained contain 627 movies, and each page only contains 50 of them (you can change the settings to obtain up to 250 movies/page, but that still won't make it till the end).

The way to automatize web scraping in these cases is to look at the URLs The one we've obtained is the following:

[https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,]

If you scroll down and click on "Next", the URL is now: https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt

Click again on "Next" and here's the new URL: https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt

The patterns are clear: our search options are in the parameters title_type, release_date and user_rating. Then, we have the start parameter, which jumps in intervals of 50, and the ref_ parameter, which takes the value of "adv_nxt".

Let's do some requests:

In [9]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests

In [10]:
# 2. url: we start with the 'second' page
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt"

In [11]:
# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [12]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")
# 4.2. check that the html code looks like it should
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Feature Film,
Released between 1990-01-01 and 1992-12-31,
User Rating at least 7.5
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/sear

Now, we'll have to build a loop where we simply replace the 51 for all the other values (jumping by 50) up until the end of the results. For simplicity, we will build manually this list of values to iterate through:



In [13]:
iterations = range(1, 631, 50)

for i in iterations:
    start_at= str(i)
    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=" + start_at + "&ref_=adv_nxt"
    print(url)

https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=1&ref_=adv_nxt
https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=51&ref_=adv_nxt
https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=101&ref_=adv_nxt
https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=151&ref_=adv_nxt
https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=201&ref_=adv_nxt
https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=251&ref_=adv_nxt
https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=301&ref_=adv_nxt
https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating

In [17]:
for i in range(2,10000):
    print (1+2)
    requests.get()

3


TypeError: get() missing 1 required positional argument: 'url'

In [ ]:
# To make it more "human", we can randomize the waiting time:
from time import sleep
from random import randint

for i in range(5):
    print(i)
    wait_time = randint(1,4)
    print("I will sleep for " + str(wait_time) + " seconds.")
    sleep(wait_time)

We will now scrape all the pages and store the response into a list - waiting a few seconds in between requests:

In [ ]:
pages = []

for i in iterations:

    # assemble the url:
    start_at= str(i)
    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=1990-01-01,1992-12-31&user_rating=7.5,&start=" + start_at + "&ref_=adv_nxt"

    # download html with a get request:
    response = requests.get(url)

    # monitor the process by printing the status code
    print("Status code: " + str(response.status_code))

    # store response into "pages" list
    pages.append(response)

    # respectful nap:
    wait_time = randint(1,4)
    print("I will sleep for " + str(wait_time) + " second/s.")
    sleep(wait_time)

Note how if you print the object pages after running the code above, you'll just see the response code messages, but the html code is still accessible and you can parse it the same way we've always done:

In [ ]:
BeautifulSoup(pages[0].content, "html.parser")

It's the moment to build the code that collects all the 631 movie titles and their synopsis in a dataframe.

#### titles

In [ ]:
# Parse just the first page, for testing purposes
soup = BeautifulSoup(pages[0].content, "html.parser")


# Paste the Selector from the first movie title copied from Chrome Dev Tools
soup.select("#main > div > div.lister.list.detail.sub-list > div > div:nth-child(1) > div.lister-item-content > h3 > a")

# Trim the selection: now it grabs all the titles
soup.select("div.lister-item-content > h3 > a")

#### synopsis

In [ ]:
# Paste the Selector from the first movie title copied from Chrome Dev Tools
soup.select("#main > div > div.lister.list.detail.sub-list > div > div:nth-child(1) > div.lister-item-content > p:nth-child(4)")

In [ ]:
# Trim the selection: now it grabs all the titles
soup.select("div.lister-item-content > p:nth-child(4)")

There are many approaches to do this. The one we'll follow is: 

- Loop through the pages we collected, parse them ("create the soup") and store the parsed pages in a list. 

- For each parsed page, select the "blocks of HTML elements" that contain all the information of each movie (the title, the synopsis and other stuff). 

- For each one of the "blocks" we collected in the previous step: 

    - Get the movie titles and store them in a list 

    - Get the synopsis and store them in a list

In [ ]:
pages_parsed = []
titles = []
synopsis = []

for i in range(len(pages)):
    # parse all pages
    pages_parsed.append(BeautifulSoup(pages[i].content, "html.parser"))
    # select only the info about the movies
    movies_html = pages_parsed[i].select("div.lister-item-content")
    # for movie, store titles and reviews into lists
    for j in range(len(movies_html)):
        titles.append(movies_html[j].select("h3 > a")[0].get_text())
        synopsis.append(movies_html[j].select("p:nth-child(4)")[0].get_text().strip())


In [ ]:
# Checking our output:
print(len(titles)) # output: 614
print(len(synopsis))  # output: 614

# Note: in your output the movie titles might be in English:
print(titles[0:3] )
print(synopsis[0:3]) 

#### Scraping presidents

Our objective is to create a dataframe with information about the presidents of the United States. To do this, we will go through this steps:

1. Scrape this [list of presidents of the United States](https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States).


In [ ]:
# 1. import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

# 2. find url and store it in a variable
url = "https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States"

# 3. download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")
# 4.2. check that the html code looks like it should
soup

In [ ]:
#mw-content-text > div.mw-parser-output > table.wikitable.sortable.jquery-tablesorter > tbody > tr:nth-child(1) > td:nth-child(3) > b > a

In [ ]:
soup.select("tbody > tr:nth-child(1) > td:nth-child(4) > b > a")

In [ ]:
# this solution is not very elegant, but works. 
# The CSS selector we copied had an "nth-child" that we could iterate 
# to find presidents, but some elements were empty, so we concatenate 
# each new element with "+" instead of appending as usual:

presidents = []

for i in range(95):
    presidents = presidents + soup.select("tbody > tr:nth-child(" + str(i) + ") > td:nth-child(4) > b > a")

# check the output:
presidents

2. Collect all the links to the Wikipedia page of each president.


In [ ]:
# we can access the links searching for the attribute "href"
# in each element
presidents[0]["href"]

In [ ]:
url = "https://en.wikipedia.org" + presidents[0]["href"]

In [ ]:
# Now, we just assemble a new request to the link
# send request
url = "https://en.wikipedia.org/" + presidents[0]["href"]
response = requests.get(url)
response.status_code

# parse & store html
soup = BeautifulSoup(response.content, "html.parser")
soup.find("table", {"class":"infobox vcard"})

3. Scrape the Wikipedia page of each president.


In this step we could very well store the whole wikipedia page for each president, or just the tiny, final pieces of information. Storing the boxes is a middle ground (we don't have too much noise but retain the flexibility of deciding later which specific elements to extract).

When sending multiple requests, remember to be respectful by spacing the requests a few seconds from each other. We will also pring the success code to monitor that everything is going well:

In [ ]:
# 2. find url and store it in a variable

presi_soups = []

for presi in presidents:
    # send request
    url = "https://en.wikipedia.org/" + presi["href"]
    response = requests.get(url)
    print(presi.get_text(), response.status_code)
    
    # parse & store html
    soup = BeautifulSoup(response.content, "html.parser")
    presi_soups.append(soup.find("table", {"class":"infobox vcard"}))
    
    # respectful nap:
    wait_time = randint(1,2)
    print("I will sleep for " + str(wait_time) + " second/s.")
    sleep(wait_time)

4. Find and store information about each president.


We extracted the 'infoboxes': now it's time to exctract especific pieces of information from them. Let's test what can we get from single presidents and then assemble a loop for all of them - as usual.

Here, we will use [the string argument](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#the-string-argument) in the find function, since wikipedia tags and classes are not always helpfulto locate. The string argument allows us to locate elements by its actual content.

In [ ]:
presi_soups[-1].select(".bday")[0].get_text()

#  -1 indicates the last president

In [ ]:
#Birthday
presi_soups[-1].find("span", {"class":"bday"}).get_text()

#Political party
presi_soups[-1].find("th", string="Political party").parent.find("a").get_text()

###second way
###presi_soup[-1].select("infobox-data > a").get_text()

#Number of sons/daughters
len(presi_soups[-1].find("th", string="Children").parent.find_all("li"))

5. Organize the information in a dataframe where we have each president as a row and each variable we collected as a column.

In [ ]:
#computer now adawy 3 billion trasnaction per second 

for i in range(1, 20):
   requests.get(url of page 2 of imdb search result )

